In [29]:
from mastodon import Mastodon
from datetime import datetime
import pytz

import os
from dotenv import load_dotenv

import requests

import pandas as pd
import numpy as np

from tabulate import tabulate

load_dotenv()

True

In [30]:
mastodon = Mastodon(
    client_id=os.environ.get("MASTODON_CLIENT_KEY"),
    client_secret=os.environ.get("MASTODON_CLIENT_SECRET"),
    access_token=os.environ.get("MASTODON_ACCESS_TOKEN"),
    api_base_url=os.environ.get("MASTODON_URL_SOCIAL")
)

In [54]:
mastodon.timeline_public(min_id='102404663597294230', limit=1)

[{'id': 102404663826836508,
  'created_at': datetime.datetime(2019, 7, 8, 7, 32, 42, 108000, tzinfo=tzutc()),
  'in_reply_to_id': None,
  'in_reply_to_account_id': None,
  'sensitive': False,
  'spoiler_text': '',
  'visibility': 'public',
  'language': 'ja',
  'uri': 'https://misskey.io/notes/7uxmu6f06g',
  'url': 'https://misskey.io/notes/7uxmu6f06g',
  'replies_count': 0,
  'reblogs_count': 0,
  'favourites_count': 0,
  'edited_at': None,
  'favourited': False,
  'reblogged': False,
  'muted': False,
  'bookmarked': False,
  'content': '<p><span>散歩して晩ごはん作ってた</span></p>',
  'filtered': [],
  'reblog': None,
  'account': {'id': 788769,
   'username': 'noizenecio',
   'acct': 'noizenecio@misskey.io',
   'display_name': 'Noizenecio',
   'locked': False,
   'bot': False,
   'discoverable': True,
   'group': False,
   'created_at': datetime.datetime(2019, 4, 15, 0, 0, tzinfo=tzutc()),
   'note': '<p><span>人間</span></p>',
   'url': 'https://misskey.io/@noizenecio',
   'avatar': 'https://fi

In [32]:
mastodon.timeline_public(min_id='104405112559589654', limit=1)

[{'id': 104405112573677116,
  'created_at': datetime.datetime(2020, 6, 25, 14, 33, 9, tzinfo=tzutc()),
  'in_reply_to_id': None,
  'in_reply_to_account_id': None,
  'sensitive': False,
  'spoiler_text': '',
  'visibility': 'public',
  'language': 'ja',
  'uri': 'https://waha.work/users/maid/statuses/104405112579043040',
  'url': 'https://waha.work/@maid/104405112579043040',
  'replies_count': 0,
  'reblogs_count': 1,
  'favourites_count': 0,
  'edited_at': None,
  'favourited': False,
  'reblogged': False,
  'muted': False,
  'bookmarked': False,
  'content': '<p>ケースも一緒</p>',
  'filtered': [],
  'reblog': None,
  'account': {'id': 945223,
   'username': 'maid',
   'acct': 'maid@waha.work',
   'display_name': 'なまきゃ',
   'locked': False,
   'bot': False,
   'discoverable': True,
   'group': False,
   'created_at': datetime.datetime(2019, 10, 17, 0, 0, tzinfo=tzutc()),
   'note': '<p>元カノに「イキんなブス」って付き合ってる時に言われたことあります。<br>それくらいブスです。</p>',
   'url': 'https://waha.work/@maid',
   'avatar': 'h

In [94]:
def binary_search_date(since_date, until_date, lower_id, upper_id):
    melbourne_tz = pytz.timezone('Australia/Melbourne')

    headers = ["ID", "Toot Date"]
    table_data = []

    while True:
        lower_toot = mastodon.timeline_public(min_id=lower_id, limit=1)[0]
        upper_toot = mastodon.timeline_public(min_id=upper_id, limit=1)[0]

        lower_created_at = lower_toot.get('created_at', datetime.utcnow()).astimezone(melbourne_tz)
        upper_created_at = upper_toot.get('created_at', datetime.utcnow()).astimezone(melbourne_tz)

        table_data = [
            [lower_id, lower_created_at.strftime('%Y-%m-%d %H:%M:%S')],
            [upper_id, upper_created_at.strftime('%Y-%m-%d %H:%M:%S')]
        ]

        print("\r" + tabulate(table_data, headers=headers, tablefmt="grid"), end='')

        if lower_created_at.date() == since_date.date() and upper_created_at.date() == until_date.date():
            break

        mid_id = (upper_id + lower_id) // 2
        toot = mastodon.timeline_public(max_id=mid_id, limit=1)[0]
        created_at = toot.get('created_at', datetime.utcnow()).astimezone(melbourne_tz)

        if created_at < since_date:
            lower_id = toot['id']
        else:
            upper_id = toot['id']

    print()  # To move to the next line in the terminal after the search is completed
    return lower_id, upper_id

In [95]:
timezone = pytz.UTC
since_date = datetime(2019, 12, 31, tzinfo=timezone)  # Fetch data since January 1, 2020
until_date = datetime(2020, 1, 2, tzinfo=timezone)  # Fetch data until Januray 1, 2020

In [97]:
lower_id = 103388556664584957   
upper_id = 103730295243282512 
binary_search_date(since_date, until_date, lower_id, upper_id)

+--------------------+---------------------+
|                 ID | Toot Date           |
+====================+=====================+
| 103388556664584957 | 2019-12-29 12:49:36 |
+--------------------+---------------------+
| 103730295243282512 | 2020-02-27 21:18:14 |
+--------------------+---------------------+
|                 ID | Toot Date           |
+====================+=====================+
| 103388556664584957 | 2019-12-29 12:49:36 |
+--------------------+---------------------+
| 103559425695099287 | 2020-01-28 17:03:56 |
+--------------------+---------------------+
|                 ID | Toot Date           |
+====================+=====================+
| 103388556664584957 | 2019-12-29 12:49:36 |
+--------------------+---------------------+
| 103473991153934297 | 2020-01-13 14:56:45 |
+--------------------+---------------------+
|                 ID | Toot Date           |
+====================+=====================+
| 103388556664584957 | 2019-12-29 12:49:36 |
+---------

(103399235850366686, 103409915161090838)